In [8]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

PATH = '/home/kai/data/kaggle/homecredit/'

# train_cleaned = pd.read_pickle(PATH + 'train_merge795.pkl')
# test_cleaned = pd.read_pickle(PATH + 'test_merge795.pkl')
print('done')

done


In [9]:
train_cleaned = pd.read_pickle(PATH+'train_original_795.pkl')
test_cleaned = pd.read_pickle(PATH+'test_original_795.pkl')

In [11]:
target = train_cleaned['TARGET']
ignore_cols = ['ORGANIZATION_TYPE', 'TARGET', 'SK_ID_CURR', 'tmp_idx']
features = [x for x in train_cleaned.columns if x not in ignore_cols]
categorical_feats = [x for x in categorical_feats if x not in ignore_cols]
train = train_cleaned[features]
test = test_cleaned[features]

In [12]:
%%time

from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import lightgbm as lgbm

lgbm_train = lgbm.Dataset(data=train,
                          label=target,
                          categorical_feature=[],#categorical_feats,
                          free_raw_data=False)

# lgbm_params = {'num_threads': 16, 'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 
#           'learning_rate': 0.01, 'num_leaves': 48, 'num_iteration': 10000, 'verbose': 100,
#           'colsample_bytree':.8, 'subsample':.9, 'max_depth':7, 'reg_alpha':.1, 'reg_lambda':.1, 
#           'min_split_gain':.01, 'min_child_weight':1}

lgbm_params ={ 'nthread' : 8,
        'n_estimators':10000,
        'learning_rate':0.02,
        'num_leaves':34,
        'colsample_bytree':0.9497036,
        'subsample':0.8715623,
        'max_depth':8,
        'reg_alpha':0.041545473,
        'reg_lambda':0.0735294,
        'min_split_gain':0.0222415,
        'min_child_weight':39.3259775,
        'silent':-1,
        'verbose':-1, }

cv_results = lgbm.cv(train_set=lgbm_train,
                     params=lgbm_params,
                     nfold=5,
                     early_stopping_rounds=150,
                     verbose_eval=100,
                     metrics=['auc'])

optimum_boost_rounds = np.argmax(cv_results['auc-mean'])
print('Optimum boost rounds = {}'.format(optimum_boost_rounds))
print('Best CV result = {}'.format(np.max(cv_results['auc-mean'])))

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:390: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:644: UserWarning: silent keyword has been found in `params` and will be ignored. Please use silent argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))


[100]	cv_agg's auc: 0.756378 + 0.00318825
[200]	cv_agg's auc: 0.77017 + 0.00239776
[300]	cv_agg's auc: 0.776049 + 0.00203482
[400]	cv_agg's auc: 0.779451 + 0.00186325
[500]	cv_agg's auc: 0.781661 + 0.00170001
[600]	cv_agg's auc: 0.783009 + 0.00161415
[700]	cv_agg's auc: 0.783753 + 0.00156213
[800]	cv_agg's auc: 0.784384 + 0.00149922
[900]	cv_agg's auc: 0.784685 + 0.00149377
[1000]	cv_agg's auc: 0.784928 + 0.00152701
[1100]	cv_agg's auc: 0.785166 + 0.00146225
[1200]	cv_agg's auc: 0.785442 + 0.00155446
[1300]	cv_agg's auc: 0.785527 + 0.00165369
[1400]	cv_agg's auc: 0.785616 + 0.00170328
[1500]	cv_agg's auc: 0.78568 + 0.00175054
[1600]	cv_agg's auc: 0.78564 + 0.00183131
Optimum boost rounds = 1457
Best CV result = 0.7856925820271229
CPU times: user 1h 55min 25s, sys: 14.1 s, total: 1h 55min 39s
Wall time: 15min 26s


In [ ]:
clf = lgbm.train(train_set=lgbm_train,
                 params=lgbm_params,
                 num_boost_round=int(optimum_boost_rounds*1.1))

""" Predict on test set and create submission """
y_pred = clf.predict(test)
out_df = pd.DataFrame({'SK_ID_CURR': test_cleaned['SK_ID_CURR'], 'TARGET': y_pred})
print('done')

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


In [19]:
print((out_df['TARGET']>0.5).sum())

NameError: name 'out_df' is not defined

In [ ]:
import matplotlib.pyplot as plt
out_df[['TARGET']].hist()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, (ax, ax1) = plt.subplots(1, 2, figsize=[15, 7])
lgbm.plot_importance(clf, ax=ax, max_num_features=30, importance_type='split')
lgbm.plot_importance(clf, ax=ax1, max_num_features=30, importance_type='gain')
ax.set_title('Importance by splits')
ax1.set_title('Importance by gain')
plt.tight_layout()
plt.show()

In [ ]:
out_df.to_csv(PATH + 'submission_795_ratio.csv', index=False)
print('done')

In [ ]:
import json

y = [x for x in zip(clf.feature_name(), clf.feature_importance('gain'))]
y.sort(key=lambda x: x[1], reverse=True)
y = [(i, j) for i, j in enumerate(y)]
y
# with open('./importance_gain.json') as f:
#     json.dump(x, f)

In [ ]:
x = [x for x in zip(clf.feature_name(), clf.feature_importance('split'))]
x.sort(key=lambda x: x[1], reverse=True)
x = [(i, j) for i, j in enumerate(x)]
x
# with open('./importance_split.json') as f:
#     json.dump(x, f)